Introduce getting started with Subgrounds and Gateway:
1. Introduce aave_v3
2. Introduce we will cover usage snapshots and lending facilities. Use GraphQL to query the data.
3. Introduce query, query df, query json
   - Usage metrics data with raw time stamp.
     - Query
     - Query_df
4. Introduce synthetic fields
   - Usage metrics data with synthetic field time stamp.
     - Lamda method
     - Helper method
5. Introduce Plotly wrapper
   - Plot simple line charts 
6. Wrap up demo
   - Gateway
   - Subgrounds


About AAVE:

Aave is a decentralized finance protocol that allows people to lend and borrow cryptocurrencies. 

Lenders earn interest by depositing digital assets into specially created liquidity pools. 

Borrowers can then use their crypto as collateral to take out a flash loan using this liquidity.

Aave V3, introducing important new risk mitigation features and improved capital efficiency

V3 was designed with a flexible architecture for increased composability, and made it possible to build a variety of innovative features on top of the protocol such as new risk management tools that provide additional security and stability. 

Examples include Supply and Borrow Caps and Risk and Listing Admins. Aave V3 also improves capital efficiency and decentralized liquidity while lowering gas fees 25 percent.

usageMetricsDailySnapshots which seems to contain daily snapshots of various usage metrics associated with the AAVE lending protocol.

- Showcase usage metrics on the graphql playgrounds

lendingProtocols: This represents a lending protocol on the AAVE V3 platform

- Showcase lending protocol on the graphql playgrounds

In [1]:
## Gateway setup

# Import subgrounds
from subgrounds import Subgrounds

# Instantiate subgrounds and insert Playgrounds API key into header
sg = Subgrounds.from_pg_key("pg-AqAFS8G3TN3Kagdgw2MrGjFvDGUgxImS")

# Insert desired subgraph id from decentralized subgraph. Find in subgraph url
aave_v3_id = "HB1Z2EAw4rtPRYVb2Nz8QGFLHCpym6ByBX6vbCViuE9F"

In [5]:
# Load subgraph using playgrounds proxy endpoint 
aave_v3 = sg.load_subgraph(f"https://api.playgrounds.network/v1/proxy/subgraphs/id/{aave_v3_id}")

In [9]:
# How to query subgraphs with subgrounds
# Subgrounds allows the same arguments as you normally use on graphql

usage_metrics_daily = aave_v3.Query.usageMetricsDailySnapshots(
    first = 10,
    orderDirection = 'desc'
)

# q_json_ex = sg.query_json([
    
#     usage_metrics_daily.timestamp,
#     usage_metrics_daily.dailyActiveBorrowers,
#     usage_metrics_daily.dailyActiveDepositors,
# ])

q_df_ex = sg.query_df([
    usage_metrics_daily.timestamp,
    usage_metrics_daily.dailyActiveBorrowers,
    usage_metrics_daily.dailyActiveDepositors,
])

In [10]:
q_df_ex

,usageMetricsDailySnapshots_timestamp,usageMetricsDailySnapshots_dailyActiveBorrowers,usageMetricsDailySnapshots_dailyActiveDepositors
0,1685115671,28,51
1,1685057351,59,60
2,1684972751,59,81
3,1684885859,52,56
4,1684799987,37,51
5,1684713515,37,83
6,1684627139,34,74
7,1684540463,43,56
8,1684453295,33,68
9,1684365839,44,75


In [ ]:
from subgrounds.subgraph import SyntheticField
from datetime import datetime

usage_metrics_daily = aave_v3.UsageMetricsDailySnapshot

# Method 1
usage_metrics_daily.d_t = SyntheticField(
    f=lambda timestamp: str(datetime.fromtimestamp(timestamp)),
    type_=SyntheticField.STRING,
    deps=usage_metrics_daily.timestamp,
)

# Method 2: This helper constructor makes it easy to convert timestamps into datetime objects.
usage_metrics_daily.datetime = SyntheticField.datetime_of_timestamp(usage_metrics_daily.timestamp)

usage_metrics_daily = aave_v3.Query.usageMetricsDailySnapshots(
    first = 100,
    orderDirection = 'desc'
)

sg.query_df([
    usage_metrics_daily.datetime,
    usage_metrics_daily.d_t,
    usage_metrics_daily.dailyActiveBorrowers,
    usage_metrics_daily.dailyActiveDepositors,
    usage_metrics_daily.dailyActiveLiquidatees,
    usage_metrics_daily.dailyActiveLiquidators,
])

Let's play with synthetic fields some more and create some interesting transformations. That we can query as if regular GraphQL fields. 

SyntheticFields can created using the constructor, allowing for much more complex transformations.

In [24]:
# Query lending protocols entitty and fields from subgraph
aave_lending = aave_v3.Query.lendingProtocols()
aave_overview = sg.query_df(aave_lending)
aave_overview.squeeze()

lendingProtocols_id                                  0xbaa999ac55eace41ccae355c77809e68bb345170
lendingProtocols_name                                                                   Aave v3
lendingProtocols_slug                                                                   aave-v3
lendingProtocols_schemaVersion                                                            2.0.1
lendingProtocols_subgraphVersion                                                          1.1.0
lendingProtocols_methodologyVersion                                                       1.0.1
lendingProtocols_network                                                                MAINNET
lendingProtocols_type                                                                   LENDING
lendingProtocols_lendingType                                                             POOLED
lendingProtocols_riskType                                                              ISOLATED
lendingProtocols_cumulativeUniqueUsers  

In [12]:
# Query lending protocols entitty and fields from subgraph
aave_lending = aave_v3.Query.lendingProtocols()
aave_overview = sg.query_df([
    aave_lending.name,
    aave_lending.type,
    aave_lending.cumulativeBorrowUSD,
    aave_lending.cumulativeDepositUSD,
    aave_lending.cumulativeLiquidateUSD,
    aave_lending.cumulativePositionCount,
    aave_lending.cumulativeProtocolSideRevenueUSD,
    aave_lending.cumulativeSupplySideRevenueUSD,
    aave_lending.cumulativeTotalRevenueUSD,
    aave_lending.cumulativeUniqueUsers,
    aave_lending.cumulativeUniqueBorrowers,
    aave_lending.cumulativeUniqueDepositors,
    aave_lending.cumulativeUniqueLiquidatees,
    aave_lending.cumulativeUniqueLiquidators,
])
aave_overview.squeeze()

lendingProtocols_name                                          Aave v3
lendingProtocols_type                                          LENDING
lendingProtocols_totalValueLockedUSD                 1444024369.600548
lendingProtocols_totalPoolCount                                     17
lendingProtocols_totalDepositBalanceUSD              1444024369.600548
lendingProtocols_totalBorrowBalanceUSD                531636986.418821
lendingProtocols_cumulativeBorrowUSD                 2994652997.847486
lendingProtocols_cumulativeDepositUSD                4147979963.583721
lendingProtocols_cumulativeLiquidateUSD                  504376.008139
lendingProtocols_cumulativePositionCount                         19468
lendingProtocols_cumulativeProtocolSideRevenueUSD        429123.059831
lendingProtocols_cumulativeSupplySideRevenueUSD         2847808.086334
lendingProtocols_cumulativeTotalRevenueUSD              3276931.146164
lendingProtocols_cumulativeUniqueUsers                            7383
lendin

In [13]:
from subgrounds.subgraph import SyntheticField

# Borrow_Deposit_Ratio: Calculate borrow deposit ratio by dividing the total borrow by the total deposit
aave_v3.LendingProtocol.borrow_dep_ratio = SyntheticField(
    lambda x, y: x/y,
    SyntheticField.FLOAT,
    [aave_v3.LendingProtocol.totalBorrowBalanceUSD,
    aave_v3.LendingProtocol.totalDepositBalanceUSD],
)

sg.query([aave_lending.totalBorrowBalanceUSD,
          aave_lending.totalDepositBalanceUSD,
          aave_lending.borrow_dep_ratio])

(531661931.6582711, 1444056498.1573937, 0.368172528108608)

In [14]:
# User Engagement Metrics: You can calculate the proportion of users who have ever borrowed,
# by dividing each of the cumulativeUnique...
# fields by cumulativeUniqueUsers

aave_v3.LendingProtocol.user_engagement_borrowed = (
    abs(aave_v3.LendingProtocol.cumulativeUniqueBorrowers)
/abs(aave_v3.LendingProtocol.cumulativeUniqueUsers)
)

sg.query_df([aave_lending.cumulativeUniqueBorrowers,
             aave_lending.cumulativeUniqueUsers,
             aave_lending.user_engagement_borrowed])

,lendingProtocols_cumulativeUniqueBorrowers,lendingProtocols_cumulativeUniqueUsers,lendingProtocols_user_engagement_borrowed
0,3560,7383,0.482189


In [18]:
# Revenue Per User: cumulativeTotalRevenueUSD divided by cumulativeUniqueUsers. 
# This metric gives an insight into how much revenue, on average, the protocol 
# generates per user.

aave_v3.LendingProtocol.revenue_per_user = (
    abs(aave_v3.LendingProtocol.cumulativeTotalRevenueUSD)
/abs(aave_v3.LendingProtocol.cumulativeUniqueUsers)
)

sg.query_df([aave_lending.revenue_per_user])

,lendingProtocols_revenue_per_user
0,443.8819


In [19]:
# Average Borrow, Deposit, and Liquidation Values: cumulativeBorrowUSD divided by cumulativeUniqueBorrowers,
# cumulativeDepositUSD divided by cumulativeUniqueDepositors, and cumulativeLiquidateUSD divided by 
# cumulativeUniqueLiquidatees will give you the average amount borrowed, deposited, and liquidated per user.

aave_v3.LendingProtocol.avg_borrow_per_user = SyntheticField(
    lambda x, y: x/y,
    SyntheticField.FLOAT,
    [aave_v3.LendingProtocol.cumulativeBorrowUSD,
    aave_v3.LendingProtocol.cumulativeUniqueBorrowers],
)

aave_v3.LendingProtocol.avg_deposit_per_user = SyntheticField(
    lambda x, y: x/y,
    SyntheticField.FLOAT,
    [aave_v3.LendingProtocol.cumulativeDepositUSD,
    aave_v3.LendingProtocol.cumulativeUniqueDepositors],
)

aave_v3.LendingProtocol.avg_liquidation_per_user = SyntheticField(
    lambda x, y: x/y,
    SyntheticField.FLOAT,
    [aave_v3.LendingProtocol.cumulativeLiquidateUSD,
    aave_v3.LendingProtocol.cumulativeUniqueLiquidatees],
)

sg.query_df([aave_lending.avg_borrow_per_user,
          aave_lending.avg_deposit_per_user,
          aave_lending.avg_liquidation_per_user])

,lendingProtocols_avg_borrow_per_user,lendingProtocols_avg_deposit_per_user,lendingProtocols_avg_liquidation_per_user
0,841196.179706,569542.765836,13631.784004


In [ ]:
from subgrounds.contrib.plotly import Figure, Scatter, Bar

# Create the Scatter trace with appropriate field paths
trace = Scatter(
    x=usage_metrics_daily.datetime,
    y=usage_metrics_daily.dailyActiveUsers,
)

# Create the Figure instance with the trace and display it
fig = Figure(
    subgrounds=sg,
    traces=trace,
    layout=dict(
        title="Daily Active Users vs Datetime",
        xaxis=dict(title="Datetime"),
        yaxis=dict(title="Daily Active Users")
    ),
)
fig.figure.show()